In [ ]:
!pip install pyshp
!pip install geopandas
!pip install --upgrade nbformat
## Ensure running conda install --channel conda-forge cartopy before
!pip install geoplot


!pip install PyBNG


!pip install convertbng

In [ ]:
import geopandas as gpd
import glob
import pandas as pd
from convertbng.util import convert_bng, convert_lonlat


Grid references for London: https://getoutside.ordnancesurvey.co.uk/guides/beginners-guide-to-grid-references/

In [ ]:
# Read SHx files
files = [
#     "SP_RoadNode.shx",
#     "TL_RoadNode.shx",
#     "SU_RoadNode.shx",
#     "TQ_RoadNode.shx",
     "NC_RoadNode.shp"

]

path = "data_storage/data_storage/oproad_essh_gb/data/"

paths = [path+file for file in files]

geo_df = pd.concat(map(gpd.read_file, paths))



#lga_gdf = gpd.read_file('data_storage/data_storage/oproad_essh_gb/data/SP_RoadNode.shx')    # Load the data using Geopandas

In [ ]:
type(geo_df)

In [ ]:
geo_df.head()

In [ ]:
geo_df.dtypes

In [ ]:
geo_df["formOfNode"].unique()

In [ ]:
geo_df["formOfNode"].value_counts()

In [ ]:
geo_df.count()

### Conversion 

In [ ]:
import plotly.express as px
import geopandas as gpd
import plotly.io as pio
pio.renderers.default = 'browser'

sample = geo_df#.sample(n=1000)

sample["longitude"], sample["latitude"] = convert_lonlat(sample.geometry.x, sample.geometry.y)

In [ ]:
fig = px.scatter_mapbox(sample,
    lat=sample.latitude,
    lon=sample.longitude,
    hover_name="formOfNode",
    zoom=14
)

fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.write_html("output/road_features.html")
fig.show()